In this tutorial we make use of `ipywidgets`. You can (in the ideal case) simply install them with executing the next cell. You will however need to restart jupyter afterwards.

In [1]:
!pip3 install --user ipywidgets
!jupyter nbextension enable --py widgetsnbextension

/bin/bash: Zeile 1: pip3: Kommando nicht gefunden.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


# Fitted space-time finite elements

In this tutorial, we consider the heat equation with Dirichlet boundary conditions and want to solve it with a fitted DG-in-time space-time finite element method. The problem is posed on the domain $\Omega = [0,1]^2$ and time in $[0,1]$. In detail, the PDE reads:

$$
\left\{
\begin{aligned}
\partial_t u - \Delta u &= f \quad \text{ in } \Omega \text{ for all } t \in [0,1],  & \\
~ \partial_{\mathbf{n}} u &=  0  \quad \text{ on } \partial \Omega, & \\
u &= u_0  \quad \text{at } t=0. & \\
\end{aligned}\right.
$$

We calculate the right-hand side $f$ such that the solution is $u = \sin(\pi t) \cdot \sin(\pi  x)^2 \cdot \sin(\pi y)^2$

In [2]:
from netgen.geom2d import unit_square
from ngsolve import *
from xfem import *
import time
from math import pi
ngsglobals.msg_level = 1

We opt for a first-order method in space and time and choose an appropriate timestep.

In [3]:
# Space finite element order
order = 1
# Time finite element order
k_t = 1
# Final simulation time
tend = 1.0
# Time step
delta_t = 1 / 32

def dt(u):
    return 1.0 / delta_t * dtref(u)

Next, we opt for a standard space-time finite element discretisation based on these parameters.

In [4]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05, quad_dominated=False))

V = H1(mesh, order=order, dirichlet=".*")
tfe = ScalarTimeFE(k_t)
st_fes = tfe * V

tnew = 0
told = Parameter(0)
t = told + delta_t * tref

gfu = GridFunction(st_fes)
u_last = CreateTimeRestrictedGF(gfu, 1)
u, v = st_fes.TnT()

The right-hand side $f$ is calculated as explained above:

In [9]:
u_exact = sin(pi * t) * sin(pi * x)**2 * sin(pi * y)**2
TimeSlider_Draw(sin(pi * tref) * sin(pi * x)**2 * sin(pi * y)**2,mesh,"u_exact", autoscale=False,min=0,max=1)
#In order to draw the function u_exact for t in [0,1] instead of t in [0, delta t], we rewrite the function

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2101-139-g7eb2ca6ee', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2…

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='tref:', max=1.0, step=0.025…

<function xfem.TimeSlider_Draw.<locals>.UpdateTime(time)>

In [10]:
coeff_f = u_exact.Diff(t) - (u_exact.Diff(x).Diff(x) + u_exact.Diff(y).Diff(y))
coeff_f = coeff_f.Compile()
TimeSlider_Draw(coeff_f,mesh,"coeff_f", autoscale=False,min=-2,max=8)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2101-139-g7eb2ca6ee', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2…

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='tref:', max=1.0, step=0.025…

<function xfem.TimeSlider_Draw.<locals>.UpdateTime(time)>

The variational formulation is derived from partial integration of the problem given above, yielding

In [11]:
dxt = delta_t * dxtref(mesh, time_order=2)
dxold = dmesh(mesh, tref=0)
dxnew = dmesh(mesh, tref=1)

In [12]:
a = BilinearForm(st_fes, symmetric=False)
a += grad(u) * grad(v) * dxt
a += u * v * dxold
a += dt(u) * v * dxt
a.Assemble()

f = LinearForm(st_fes)
f += coeff_f * v * dxt
f += u_last * v * dxold

Finally, the problem is solved in a time-stepping manner:

In [17]:
scene = Draw(u_last, mesh,"u", autoscale=False,min=0,max=1)
u_last.Set(fix_tref(u_exact, 0))
told.Set(0.)
while tend - told.Get() > delta_t / 2:
    f.Assemble()
    gfu.vec.data = a.mat.Inverse(st_fes.FreeDofs(), "umfpack") * f.vec
    RestrictGFInTime(spacetime_gf=gfu, reference_time=1.0, space_gf=u_last)
    l2error = sqrt(Integrate((u_exact - gfu)**2 * dxnew, mesh))
    scene.Redraw()
    time.sleep(0.1)
    told.Set(told.Get() + delta_t)
    print("\rt = {0:12.9f}, L2 error = {1:12.9e}".format(told.Get(), l2error))

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2101-139-g7eb2ca6ee', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2…

t =  0.031250000, L2 error = 2.084111112e-04
t =  0.062500000, L2 error = 4.693640620e-04
t =  0.093750000, L2 error = 7.332587318e-04
t =  0.125000000, L2 error = 9.951800950e-04
t =  0.156250000, L2 error = 1.250524467e-03
t =  0.187500000, L2 error = 1.495600082e-03
t =  0.218750000, L2 error = 1.727320046e-03
t =  0.250000000, L2 error = 1.943033189e-03
t =  0.281250000, L2 error = 2.140420860e-03
t =  0.312500000, L2 error = 2.317443093e-03
t =  0.343750000, L2 error = 2.472314308e-03
t =  0.375000000, L2 error = 2.603495538e-03
t =  0.406250000, L2 error = 2.709695164e-03
t =  0.437500000, L2 error = 2.789873426e-03
t =  0.468750000, L2 error = 2.843247981e-03
t =  0.500000000, L2 error = 2.869298937e-03
t =  0.531250000, L2 error = 2.867772490e-03
t =  0.562500000, L2 error = 2.838682660e-03
t =  0.593750000, L2 error = 2.782310868e-03
t =  0.625000000, L2 error = 2.699203252e-03
t =  0.656250000, L2 error = 2.590165731e-03
t =  0.687500000, L2 error = 2.456256925e-03
t =  0.718